### **Dataset**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
class StockPricePredictor:
    def __init__(self, csv_file, time_step=2):
        self.df = pd.read_csv(csv_file)
        self.time_step = time_step
        self.scaler = StandardScaler()
        self.model = None

    def preprocess_data(self):
        # Convert 'Date' to datetime and extract year, month, day
        self.df['Date'] = self.df['Date'].astype(str).str.split(' ').str[0]
        self.df['Date'] = pd.to_datetime(self.df['Date'], errors='coerce')
        self.df['year'] = self.df['Date'].dt.year
        self.df['month'] = self.df['Date'].dt.month
        self.df['day'] = self.df['Date'].dt.day
        self.df.drop(columns=['Date', 'Company'], inplace=True)

        # Split features and target
        X = self.df[['Open', 'High', 'Low', 'Volume']]
        Y = self.df['Close'].values

        # Scale the features
        X_scaled = self.scaler.fit_transform(X)
        return X_scaled, Y

    def create_sequences(self, data, target):
        X_seq, y_seq = [], []
        for i in range(len(data) - self.time_step):
            X_seq.append(data[i:(i + self.time_step)])
            y_seq.append(target[i + self.time_step])
        return np.array(X_seq), np.array(y_seq)

    def build_model(self, input_shape):
        self.model = Sequential()
        self.model.add(LSTM(100, return_sequences=True, input_shape=input_shape))
        self.model.add(Dropout(0.3))
        self.model.add(LSTM(100, return_sequences=False))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(50))
        self.model.add(Dense(1))
        self.model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
                           loss='mean_squared_error')
        self.model.summary()

    def train_model(self, X_seq, y_seq):
        # Use only the first 20% of the data for faster training
        X_train = X_seq[:int(0.2 * len(X_seq))]
        y_train = y_seq[:int(0.2 * len(y_seq))]

        # Create a TensorFlow dataset
        train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
        train_dataset = train_dataset.shuffle(buffer_size=512).batch(64).prefetch(tf.data.experimental.AUTOTUNE)

        # Train the model
        history = self.model.fit(train_dataset, epochs=10)

        # Plot the loss
        plt.plot(history.history['loss'])
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.show()

    def make_prediction(self, input_data):
        # Reshape the input to match model's expected shape
        reshaped_input = np.tile(input_data, (self.time_step, 1))
        reshaped_input = reshaped_input.reshape((1, self.time_step, reshaped_input.shape[1]))

        # Make a prediction
        predictions = self.model.predict(reshaped_input)
        return predictions

# Example usage
if __name__ == "__main__":
    # Initialize the class
    predictor = StockPricePredictor('stock_details_5_years.csv')

    # Preprocess the data
    X_scaled, Y = predictor.preprocess_data()

    # Create sequences
    X_seq, y_seq = predictor.create_sequences(X_scaled, Y)

    # Build and train the model
    input_shape = (X_seq.shape[1], X_seq.shape[2])
    predictor.build_model(input_shape)
    predictor.train_model(X_seq, y_seq)

    # Example prediction
    input_data = np.array([[54.176498, 55.007500, 54.099998, 31004000]])  # Shape is (1, 4)
    prediction = predictor.make_prediction(input_data)
    print("Prediction:", prediction)
